In [1]:
import sionna.rt
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

import mitsuba as mi
import drjit as dr
import numpy as np
import torch

In [2]:
# scene = load_scene(filename="cubes/cubes.xml")
scene = load_scene(filename="grid/row0_col0.xml")
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")

scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          polarization="V",
                          pattern="iso")


In [3]:
# scene.preview()

In [4]:
mi_scene = scene.mi_scene

In [5]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []
for s in shapes:
    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

# print(face_averages.shape)
# print(face_averages)

In [6]:
i = 0
for face_avg in face_averages[3:6]:
    # print(face_avg)
    rx = Receiver(name=f"rx{i}", position=face_avg)
    scene.add(rx)
    i += 1

# rx = Receiver(position=face_averages)
# scene.add(rx)

In [7]:
offset_normals = face_normals + face_averages

i = 0
for face_n in offset_normals:
    # print(face_n)
    tx = Transmitter(name=f"tx{i}", position=face_n)
    scene.add(tx)
    i += 1
# tx = Transmitter(position=offset_normals)
# scene.add(tx)

scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [8]:
face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
print(face_averages)
print(face_normals)


[[-1605.9, -1563.7, 4.66667],
 [-1603.18, -1551.13, 6.33333],
 [-1600.08, -1578.17, 4.66667],
 .. 107 skipped ..,
 [-1436.18, -1475.7, 7.13571],
 [-1336.33, -1382.64, 4],
 [-1337.64, -1383.64, 4]]
[[-0.977329, 0.211727, 1.23607e-09],
 [-0.977329, 0.211727, 0],
 [-0.217247, -0.976117, 9.0789e-09],
 .. 107 skipped ..,
 [0, 0, 1],
 [0, 0, 1],
 [0, -0, 1]]


In [9]:
# Generate rays to be shot from each surface in 180 cone

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
ray_per_source = 1000

sources = face_averages + (face_normals * 0.001)
rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
print(norms_expanded)
# shouldn't this be positive dot prod???
active = dr.dot(rays.d, norms_expanded) > 0
print(active)

Ray[
  o=[[-1605.9, -1563.7, 4.66667],
     [-1605.9, -1563.7, 4.66667],
     [-1605.9, -1563.7, 4.66667],
     [-1605.9, -1563.7, 4.66667],
     [-1605.9, -1563.7, 4.66667],
        112990 skipped ..,
     [-1337.64, -1383.64, 4.001],
     [-1337.64, -1383.64, 4.001],
     [-1337.64, -1383.64, 4.001],
     [-1337.64, -1383.64, 4.001],
     [-1337.64, -1383.64, 4.001]],
  d=[[0, 0, 1],
     [-0.0466353, -0.0427217, 0.997998],
     [0.00781566, 0.0890555, 0.995996],
     [0.0665843, -0.0868474, 0.993994],
     [-0.12437, 0.0219993, 0.991992],
        112990 skipped ..,
     [0.118513, -0.0436636, -0.991992],
     [-0.101274, -0.0414674, -0.993994],
     [0.0381213, 0.0808625, -0.995996],
     [0.0187565, -0.0604002, -0.997998],
     [-0, 0, -1]],
  maxt=[3.40282e+38],
  time=[0, 0, 0, 0, 0,    112990 skipped .., 0, 0, 0, 0, 0],
  wavelengths=[]
]
[[-0.977329, 0.211727, 1.23607e-09],
 [-0.977329, 0.211727, 1.23607e-09],
 [-0.977329, 0.211727, 1.23607e-09],
 .. 112994 skipped ..,
 [0, -0,

In [10]:
interactions = mi_scene.ray_intersect_preliminary(rays, active=active)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
b=dr.select(interactions.is_valid(), interactions.t, dr.cuda.Float32(-1)).numpy()

# for i in range(len(a)):
#     print(a[i], b[i])



# for i, b in enumerate(a):
#     print(b, interactions.prim_index[i])


# Need to check to make sure ray is intersecting with the correct side of plane!!!!!!

In [11]:

sets = [set() for _ in range(count)]

for i in range(len(a)):
    vis = a[i]
    if vis < count:
        sets[i // ray_per_source].add(vis)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

scene

[-1605.9003   -1563.698        4.666667] {50}
[-1603.1777    -1551.1306        6.3333335] set()
[-1600.0784   -1578.167        4.666667] {16, 17, 56}
[-1604.3506    -1577.2162        6.3333335] {16, 17, 56}
[-1590.0521   -1556.9369       4.666667] {4, 5, 6, 7, 15, 49}
[-1592.9291    -1568.0273        6.3333335] {4, 5, 6, 7, 17}
[-1596.0286   -1540.991        4.666667] {4, 5, 6, 7}
[-1591.6019    -1543.4187        6.3333335] {36, 4, 5, 6, 7}
[-1597.2014 -1567.0765     8.    ] {4, 5, 6, 7}
[-1598.751  -1553.5583     8.    ] {4, 5, 6, 7}
[-1610.876     -1603.8127        4.3333335] {57, 2, 3}
[-1609.0205   -1594.8369       6.166667] {57, 2, 3}
[-1607.1688    -1615.6318        4.3333335] {57}
[-1609.9502   -1614.2102       6.166667] {57}
[-1600.3558    -1599.4587        4.3333335] {4, 5, 6, 7}
[-1602.3716   -1608.2562       6.166667] {4, 5, 6, 7, 58}
[-1604.2234    -1587.4613        4.3333335] {2, 3, 4, 5, 6, 7}
[-1601.2816   -1589.0613       6.166667] {2, 3, 4, 5, 6, 7}
[-1605.1531 -1606.8

interactions is 1d array num faces x rays per source long
reshape

In [12]:
scene.preview()

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, matrixWorldNeedsUpda…